In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import combinations
import string
from collections import Counter, defaultdict
import re

In [2]:
path_connected_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\"
path_temp_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\temporaryData\\"
year = "2007"

full_path = f"{path_connected_data}{year}_gptDataframe.csv"
grouped_path = f"{path_temp_data}{year}_grouped.csv"
grouped_validated_path = f"{path_temp_data}full_validation.csv"
word_path = f"{path_temp_data}institution_wordbank.csv"
ungrouped_path = f"{path_temp_data}{year}_unclassified_subinstitutions.csv"

pd.options.display.max_rows = 400

In [3]:
fulldf = pd.read_csv(full_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\2007_gptDataframe.csv'

In [ ]:
#define keywords
keywords = {
    "Arts and Sciences": ["arts and science", "letters and science", "arts, sciences, and letters", "arts, sciences", "arts & sciences", "letters, sciences", "arts and letters", "arts & letters"],
    "Engineering": ["engineering", "mines", "aerospace", "aeronautics", "polytechnic studies", "aviation", "applied science", "computational science"],
    
    "Computer Science": ["computer science", "computer,", "software engineering", "and computer", "computing"],
    "Data Science": ["data science", "artificial intelligence", "machine learning"],
    "Technology": ["technology"],
    "Information": ["information science", "information management", "information systems", "information stud", "library science", "and information", "library stud", "informatics", "school of information", 
                    "information and", "information school"],
    "Archictecure": ["architecture", "architectural", "urban design", "urban planning", "environmental design", "environment and design", "design, construction", "school of construction"],
    

    "Behavioral Science": ["psychology", "psychological", "human science", "human ecology", "behavioral", "human and", "childhood stud"],
    "Social Sciences": ["social science", "social and", "social ecology"],
    "Natural Sciences": ['natural science', "mathematics", "chemistry", "biology", "mathematical", "physical science", "life science", "biological science", 
                        "biochemistry", "natural and", "basic", "college of science", "school of science", ", science, and", ", science and", "physics", "college of the sciences"],
    "Environmental and Earth Science": ["environmental science", "forestry", "natural resource", "college of forest", 
                              "school of forest", "school of environment ", "environmental and", "geology", "oceanography", "earth science", "geophysics", "seismology", "volcanology", "earth and" 
                              "paleontology", "mineral science", "geoscience", "earth science", "earth studies", "atmospheric", "college of the environment", "school of ecology", "school of environment"],
    "Marine Science": ["marine science", "marine stud", "ocean science", "oceanography", "oceanographic", "ocean and"],

    
    "Business": ["business", "commerce", "mba", "college of economics", "and economics", "finance", "entrepreneurship", "school of management", "and management", "accounting", "agribusiness", "economic,", "accountancy", "schools of management", "college of management"],
    "Hospitality": ["hospitality", "hotel administration", "hotel management", "restaurant management"],
    "Education": ["college of education", "school of education", "teaching", "teach", "curriculum development", "human development", "and education", ", education and"],
    "Criminal Justice": ["criminal justice", "criminology", "criminal", "forensics"],


    "Law": ["law", "legal studies"],
    "Policy and Affairs": ["public policy", "international policy", "public affairs", "international affairs", "environmental affairs", "urban affairs", "metropolitan affairs", "international service", "leadership studies", 
                           "diplomacy", "diplomat", "international studies", "public administration", "urban policy", "global studies", "international relations", "social policy", "peace studies", "global management", "policy studies",
                           "policy, planning", "school of leadership", "policy science"],
    

    "Agriculture": ["agriculture", "agricultural", "agribusiness"],
    "Family and Consumer Science": ["family and", "home economics", "consumer science", "family science", "home", "family studies"],
    "Culinary": ["culinary", "cooking", "gastronomy"],


    "Journalism": ["journalism"],
    "Communication": ["communication"],
    "Liberal Arts": ["liberal art", "liberal studies", "interdisciplinary studies", "general studies", "liberal and", "individualized study"],
    "Humanities": ["humanities", "history", "historical", "philosophy", "literature", "political"],
    "Fine Arts": ["fine arts", "music", "theater", "creative", "theatre", "film", "performing art", "the arts", "design art", "dramatic arts", "dance",
                   "visual arts", "school of drama", "recording arts", "ceramics", "fine and applied art", "school of design", "art and design", "fine", 
                   "cinema", "art ", "art, ", "studio art", "media art", "arts and performance", "drama", "arts and media", "textiles", "applied arts"],
    "Language and Culture": ["languages", "lingusitics", "cultures", "translational studies", "intercultural studies", "ethnic studies", "hebraic studies", "cultural stud", "asian studies", "culture ", " hawaiian knowledge", 
                             "jewish", "asian and pacific studies"],


    "Nursing": ["nursing"],
    "Physical Therapy and Nutrition": ["physical therapy", "nutrition", "physiotherapy", "nutritional therapy", "rehab", "human performance"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine", "human medicine"],
    "Pharmacy": ["pharmacy", "pharmacology"],
    "Dentistry" : ["dentistry", "dental", "denistry"],
    "Optometry" : ["optometry", "optics", "optical science"],
    "Veterinary" : ['veterinary'],
    "Audiology": ["audiology"],
    "Gerontology": ["gerontology"],
    "Pathology": ["pathology"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome", "occupational therapy"],


    "Religious Studies": ["religious", "theology", "divinity", "biblical", "theological", "religion", "seminary", "religous", "christ college", "talmudic", "torah", "quran", "bible"],
    "Social Work": ["social work", "social welfare", "community service", "human service", "social service", "public service", "social administration"],
    

    "Graduate": ["graduate school", "graduate studies", "graduate college", "college of graduate", "school of graduate",  "advancing", "advanced study", "graduate division", "graduate and professional studies", "doctoral", "graduate study", "graduate "],
    "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning", "adult learning",
                           "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "distance learning", "continuous education", "extended university"],
    "Professional Studies": ["professional studies", "professional development", "professional education", "school of professions", "professional campus"],
    "Administration": ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", 
                       "university librar", "information service", "information access", "library access", "residence", "academic affairs", "library affairs", "academic succes", "first year student",
                       "undergraduate studies", "undergraduate education"],
    "Honor College": ["honor college", "honors college", "scholar program"],

    "Satellite Campus": ["campus"],
    "Foundation": ["foundation"],
    "Boards": ["board of", "regents", "trustees", "council of"],
    "Hospital": ["hospital"],
    "University College": ["university college"],
    "Error": []
}

In [ ]:
keyword2category = {k:c for c, ks in keywords.items() for k in ks}
#dictionary of "school of" + category and #college of" + category
schoolwords = {"school of "+s.lower():s for s in keywords.keys()}
schoolwords.update({"college of "+s.lower():s for s in keywords.keys()})
schoolwords['school of medicine'] = "Medical"
schoolwords['school of management'] = "Business"
schoolwords['college of management'] = "Business"
schoolwords['school of economics'] = "Business"
schoolwords['teachers college'] = "Education"
schoolwords["college of science"] = "Natural Sciences"
schoolwords["school of science"] = "Natural Sciences"
schoolwords["college of sciences"] = "Natural Sciences"
schoolwords["school of sciences"] = "Natural Sciences"
schoolwords["college of design"] = "Fine Arts"
schoolwords["school of design"] = "Fine Arts"
schoolwords["college of art"] = "Fine Arts"
schoolwords["school of art"] = "Fine Arts"
schoolwords["college of arts"] = "Fine Arts"
schoolwords["school of arts"] = "Fine Arts"
schoolwords["library"] = "Administration"
schoolwords["libraries"] = "Administration"
schoolwords["students"]  = "Administration"
schoolwords["university library"] = "Administration"

In [ ]:
#create table of unique institution + subinstitutions
unique_combinations = set()
all_subinstitutions = []
for index, row in fulldf.iterrows():
    institution = row["Institution"]
    subinstitution = row["SubInstitution"]
    combination = (institution, subinstitution)
    if combination not in unique_combinations and pd.notnull(subinstitution):
        unique_combinations.add(combination)
        all_subinstitutions.append(combination)

unique_df = pd.DataFrame(all_subinstitutions, columns=["Institution", "SubInstitution"])
unique_df["Category"] = ""
print(unique_df.shape)

(4206, 3)


In [ ]:
#now find the high-confidence classifcations. e.g. "school of + category"
#Checking for equality, not substring
unique_df["Category"] = ""
high_confidence = pd.DataFrame(columns=["Institution", "SubInstitution", "Category"])
uncategorized_df = pd.DataFrame(columns=["Institution", "SubInstitution", "Category"])

for index, row in unique_df.iterrows():
    subinstitution = row["SubInstitution"].lower()
    for key, value in schoolwords.items():
        if key == subinstitution:
            if(row["Category"] == ""):
                unique_df.at[index, "Category"] += str(value)
                
            else:
                unique_df.at[index, "Category"] += str(value) + ", "
    if unique_df.at[index, "Category"] != "":
        high_confidence = pd.concat([high_confidence, unique_df.loc[[index]]], ignore_index=True)
    else:
        uncategorized_df = pd.concat([uncategorized_df, unique_df.loc[[index]]], ignore_index=True)
        
print(high_confidence.shape)


(1314, 3)


In [ ]:
#count what category each of the high-confidence classifications are in
category_counter = Counter()
for category_list in high_confidence["Category"]:
    categories = [cat.strip() for cat in category_list.split(",") if cat.strip()]
    category_counter.update(categories)

total_categories = sum(category_counter.values())
category_fractions = {category: count / total_categories for category, count in category_counter.items()}
sorted_category_fractions = dict(sorted(category_fractions.items(), key=lambda item: item[1], reverse=True))

print("Percentage of each category: (high conf):")
for category, fraction in sorted_category_fractions.items():
    print(f"{category}: {fraction:.4f}")

Percentage of each category: (high conf):
Education: 0.1788
Arts and Sciences: 0.1385
Business: 0.0928
Nursing: 0.0913
Engineering: 0.0883
Law: 0.0738
Natural Sciences: 0.0434
Medical: 0.0411
Pharmacy: 0.0411
Liberal Arts: 0.0396
Social Work: 0.0289
Fine Arts: 0.0251
Dentistry: 0.0228
Administration: 0.0160
Professional Studies: 0.0145
Communication: 0.0107
Technology: 0.0107
Agriculture: 0.0099
Humanities: 0.0084
Social Sciences: 0.0061
Optometry: 0.0061
Journalism: 0.0038
Criminal Justice: 0.0030
Marine Science: 0.0023
Information: 0.0023
Computer Science: 0.0008


In [ ]:
#categorize rest of the subinstitutions, including those who fall under multiple categories
medium_confidence = pd.DataFrame(columns=["Institution", "SubInstitution", "Category"])
uncategorized_df["Category"] = ""

for index, row in uncategorized_df.iterrows():
    institution = row["Institution"].title()
    subinstitution = row["SubInstitution"].title()
    #check every word in the subinstitution
    matched = False
    for key, value in keyword2category.items():
        # print(word, key)
        if key.title() in subinstitution:
            if not matched:
                uncategorized_df.at[index, "Category"] += str(value)
                matched = True
            elif matched and str(value) not in uncategorized_df.at[index, "Category"]:
                uncategorized_df.at[index, "Category"] += ", " + str(value)
    if uncategorized_df.at[index, "Category"] != "":
        medium_confidence = pd.concat([medium_confidence, uncategorized_df.loc[[index]]], ignore_index=True)
        unique_df.loc[unique_df['SubInstitution'].str.title() == subinstitution, 'Category'] = uncategorized_df.at[index, 'Category']
    elif institution in subinstitution and uncategorized_df.at[index, "Category"] == "" and institution != subinstitution:
        uncategorized_df.at[index, "Category"] = "Satellite Campus"
        medium_confidence = pd.concat([medium_confidence, uncategorized_df.loc[[index]]], ignore_index=True)
        unique_df.loc[unique_df['SubInstitution'].str.title() == subinstitution, 'Category'] = uncategorized_df.at[index, 'Category']

In [ ]:
#cleanup incorrect classifications
for index, row in medium_confidence.iterrows():
    category = row["Category"]
    subinstitution = row["SubInstitution"].title()
    institution = row["Institution"].title()
    word_list = list(subinstitution.split())
    art_count = word_list.count("Arts")
    engineering_count = word_list.count("Engineering")

    if "Humanities" in category and "Arts and Sciences" in category and "Fine Arts" in category and "Fine Arts" in subinstitution and art_count == 1:
        medium_confidence.at[index, "Category"] = category.replace("Arts and Sciences", "").strip(", ")
    if "Communication Arts And Science" in subinstitution and "Arts and Sciences" in category:
        medium_confidence.at[index, "Category"] = category.replace("Arts and Sciences", "").strip(", ")
    if "Arts and Sciences" not in category and "Fine Arts" not in category and "Liberal Art" not in subinstitution and art_count == 1:
         medium_confidence.at[index, "Category"] += ", Fine Arts"
    if "Administration" in category:
        medium_confidence.at[index, "Category"] = "Administration"
    if "Arts," in subinstitution and "Arts And Sciences" not in category and "Liberal Arts" not in category and "Fine Arts" not in category:
        medium_confidence.at[index, "Category"] += ", Fine Arts"
    if "And Science" in subinstitution and "Arts and Sciences" not in category and "Natural Sciences" not in category:
        medium_confidence.at[index, "Category"] += ", Natural Sciences"
    if "Liberal Arts" in category and "Arts and Sciences" in category:
        medium_confidence.at[index, "Category"] = category.replace("Liberal Arts", "").strip(", ")
    if "Software Engineering" in subinstitution and "Engineering" in category and engineering_count == 1:
        medium_confidence.at[index, "Category"] = category.replace("Engineering", "").strip(", ")
    if "Hospitality" in subinstitution and "Hospital" in category:
        medium_confidence.at[index, "Category"] = category.replace("Hospital,", "").strip(", ")
    if "Foundation School" in subinstitution:
        medium_confidence.at[index, "Category"] = category.replace("Foundation", "").strip(", ")

In [ ]:
#count how much each category appears for the medium confidence 
category_counter = Counter()
for category_list in medium_confidence["Category"]:
    categories = [cat.strip() for cat in category_list.split(",") if cat.strip()]
    category_counter.update(categories)


total_categories = sum(category_counter.values())
category_fractions = {category: count / total_categories for category, count in category_counter.items()}
sorted_category_fractions = dict(sorted(category_fractions.items(), key=lambda item: item[1], reverse=True))

print("\nCategory Percentages (all classified):")
for category, fraction in sorted_category_fractions.items():
    print(f"{category}: {fraction:.4f}")


Category Percentages (all classified):
Business: 0.1018
Graduate: 0.0930
Health Science: 0.0737
Fine Arts: 0.0641
Natural Sciences: 0.0476
Education: 0.0457
Engineering: 0.0451
Arts and Sciences: 0.0383
Foundation: 0.0346
Humanities: 0.0261
Social Sciences: 0.0258
Technology: 0.0255
Social Work: 0.0252
Policy and Affairs: 0.0224
Behavioral Science: 0.0221
Archictecure: 0.0216
Medical: 0.0216
Environmental and Earth Science: 0.0199
Administration: 0.0196
Law: 0.0182
Nursing: 0.0176
Communication: 0.0176
Agriculture: 0.0170
Continued Studies: 0.0162
Satellite Campus: 0.0156
Information: 0.0147
Liberal Arts: 0.0125
Computer Science: 0.0108
Religious Studies: 0.0096
Professional Studies: 0.0088
University College: 0.0085
Honor College: 0.0085
Veterinary: 0.0077
Pharmacy: 0.0065
Dentistry: 0.0062
Journalism: 0.0060
Boards: 0.0043
Physical Therapy and Nutrition: 0.0043
Language and Culture: 0.0037
Hospitality: 0.0028
Family and Consumer Science: 0.0026
Hospital: 0.0026
Marine Science: 0.001

In [ ]:
#categorize remaining "others"

uncategorized_df["Category"] = ""
na_df = unique_df[unique_df["Category"] == ""].copy()
na_df["Category"] = "Other"
    
institution_list = list(set(fulldf["Institution"].tolist()))

for index, row in na_df.iterrows():
    category = row["Category"]
    subinstitution = row["SubInstitution"].title()
    institution = row["Institution"].title()
    word_list = list(subinstitution.split())
    institution_appears = any(substring in subinstitution for substring in institution_list)
    #remaining schools with "library" are not library sciences/information science anymore
    if "Library" in subinstitution:
        na_df.at[index, "Category"] = "Administration"

In [ ]:
combination_counter = Counter()
subinstitution_texts = {}

for index, row in medium_confidence.iterrows():
    category_list = row["Category"]
    subinstitution = row["SubInstitution"]
    categories = sorted([cat.strip() for cat in category_list.split(",") if cat.strip()])

    if len(categories) > 1:
        for size in range(2, len(categories) + 1):
            for c in combinations(categories, size):
                #convert combination to a tuple to use it as dict key
                c_tuple = tuple(c)
                combination_counter[c_tuple] += 1
                
                if c_tuple not in subinstitution_texts:
                    subinstitution_texts[c_tuple] = []
                subinstitution_texts[c_tuple].append(subinstitution)

sorted_combinations = sorted(combination_counter.items(), key=lambda item: item[1], reverse=True)

#Displays combinations incorrectly e.g.Category = "Business, Humanaties, Social Sciences" will not only display as that combination,
#but also display as "Business, humanities" and "Business, Social Sciences", etc, so don't use
#to gauge accuracy of classification
print("Category combinations:")
for combination, count in sorted_combinations:
    print(f"{combination}: {count}")
    print("SubInstitutions:", subinstitution_texts[combination])
    print()

Category combinations:
('Fine Arts', 'Humanities'): 46
SubInstitutions: ['School of Fine Arts and Humanities', 'College of Humanities and Fine Arts', 'College of Arts and Humanities', 'College of Arts and Humanities', 'College of Arts, Humanities, and Social Sciences', 'College of Arts, Humanities and Social Sciences', 'College of Humanities and the Arts', 'School of Arts and Humanities', 'Languages, Literature, Arts and Education', 'College of Arts and Humanities', 'School of Arts and Humanities', 'College of Architecture, Arts and Humanities', 'College of Arts and Humanities', 'School of Arts, Humanities and Social Sciences', 'College of Arts and Humanities', 'School of Humanities, Arts and Social Sciences', 'College of Arts and Humanities', 'College of Arts and Humanities', 'College of Humanities and Fine Arts', 'College of Arts, Humanities and Social Sciences', 'College of Humanities, Fine and Performing Arts', 'School of Arts and Humanities', 'Arts and Humanities', 'Sybil B. Harri

In [ ]:
# Combine the DataFrames
combined_df = pd.concat([medium_confidence, na_df, high_confidence], ignore_index=True)
sorted_combined_df = combined_df.sort_values(by="Category")

In [ ]:
for index, row in sorted_combined_df.iterrows():
    subinstitution = row["SubInstitution"].title()
    category = row["Category"].title()
    admin_list = ["Services", "Programs", "Resources", "Freshman", "Freshmen", "Association", "Plans", "Planning", "Outreach", "Faculty"]
    if any(s in subinstitution for s in admin_list):
        sorted_combined_df.at[index, "Category"] = "Administration"
sorted_combined_df = combined_df.sort_values(by="Category")

In [ ]:
category_counter = Counter()
for category_list in combined_df["Category"]:
    categories = [cat.strip() for cat in category_list.split(",") if cat.strip()]
    category_counter.update(categories)

sorted_category = dict(sorted(category_counter.items(), key=lambda item: item[1], reverse=True))
print("\nCategory counts")
for category, number in sorted_category.items():
    print(f"{category}: {number}")


Category counts
Business: 481
Education: 396
Graduate: 328
Arts and Sciences: 317
Engineering: 275
Health Science: 260
Fine Arts: 259
Natural Sciences: 225
Other: 183
Nursing: 182
Law: 161
Medical: 130
Social Work: 127
Foundation: 122
Technology: 104
Humanities: 103
Social Sciences: 99
Liberal Arts: 96
Administration: 95
Policy and Affairs: 79
Behavioral Science: 78
Pharmacy: 77
Communication: 76
Archictecure: 76
Agriculture: 73
Environmental and Earth Science: 70
Continued Studies: 57
Satellite Campus: 55
Information: 55
Dentistry: 52
Professional Studies: 50
Computer Science: 39
Religious Studies: 34
University College: 30
Honor College: 30
Veterinary: 27
Journalism: 26
Boards: 15
Physical Therapy and Nutrition: 15
Language and Culture: 13
Hospitality: 10
Optometry: 10
Family and Consumer Science: 9
Hospital: 9
Marine Science: 9
Criminal Justice: 6
Pathology: 2
Culinary: 1
Audiology: 1
Gerontology: 1


In [ ]:
#things not classified as education but has education in
for index, row in sorted_combined_df.iterrows():
    subinstitution = row["SubInstitution"].title()
    category = row["Category"].title()
    if "Education" in subinstitution and "Education" not in category and "Continued Studies" not in category:
        print(subinstitution)

Undergraduate Education
Undergraduate Education
Enrollment Services And General Education
Center For Undergraduate Education
Undergraduate Education
Hudson Institute Of Entrepreneurship And Executive Education
California Lutheran Educational Foundation
Cal State East Bay Educational Foundation
University Of Baltimore Educational Foundation
Pennsylvania State System Of Higher Education Foundation
School Of Physical Education
School Of Physical Education
Education And Health Sciences
School Of Health, Physical Education And Recreation
Azrieli Graduate School Of Jewish Education And Administration
Sophie Davis School Of Biomedical Education/Cuny Medical School
School Of Graduate Medical Education
Education
Elementary Schools Department, National Catholic Educational Association
General Education
School Of Community Education
School Of Professional Education
College Of Professional Education
Graduate School Of Religion And Religious Education


In [ ]:
for index, row in sorted_combined_df.iterrows():
    subinstitution = row["SubInstitution"].title()
    category = row["Category"].title()
    if "Art" in subinstitution and "Arts And Sciences" not in category and "Liberal Arts" not in category and "Fine Arts" not in category:
        print(subinstitution)

Arthur J. Kania School Of Management
College Of Communication Arts And Sciences
Arthur A. Dugoni School Of Dentistry
College And Graduate School Of Art
School Of Arts And Professions
Tyler School Of Art
Herron School Of Art


In [ ]:
for index, row in sorted_combined_df.iterrows():
    subinstitution = row["SubInstitution"].title()
    category = row["Category"].title()
    for key in keywords.keys():
        if key in subinstitution and key not in category:
            print(key, ": ", subinstitution)

Education :  Undergraduate Education
Education :  Undergraduate Education
Information :  Library And Information Services
Education :  Enrollment Services And General Education
Information :  Information Services
Information :  School Of University Studies And Academic Information Services
Information :  Library And Information Access (Interim)
Information :  Information Resources And University Libraries
Education :  Center For Undergraduate Education
Education :  Undergraduate Education
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts, Sciences And Technologies
Liberal Arts : 

In [ ]:
for index, row in sorted_combined_df.iterrows():
    subinstitution = row["SubInstitution"].title()
    category = row["Category"].title()
    if category == "Other":
        print(subinstitution)

East Carolina University
Libraries And Academic Information Resources
Newcomb-Tulane Undergraduate College
Casper College
Undergraduate College
California Centers
Outreach School
Cincinnati Center
Summit College
Miami Center
Rio Grande College
College At Wise
Wayne College
Community And Technical College
University Learning Collaborative
University Of Wisconsin La Crosse
The Erickson School
President And Vice Chancellor, Health Affairs
Professional Schools And Colleges
Rockefeller College
Buffalo State College
School Of The Professions
Wake Forest College
College
Education
School Of Arts And Professions
College
Stony Brook
Thomas Wheeler College Of Student Studies
Peabody College
School Of Systems And Enterprises
Tyler School Of Art
College Of Rural And Community Development
College
University Studies
International And Area Studies
Extension
Usc Beaufort
School Of Travel Industry Management
Uic
Usc Aiken
University Of Puget Sound
College Of Applied Life Studies
International Programs
O

In [ ]:
category_counter = Counter(sorted_combined_df['Category'])
num_other = category_counter['Other']
total = len(sorted_combined_df)
print("Remaining unclassified (pre matching): ", 100*(num_other/total), "%")

Remaining unclassified (pre matching):  4.3509272467903 %


In [ ]:
validated_df = pd.read_csv(grouped_validated_path)
validated_df.sort_values(by=["Category", "Institution"])
validated_subinstitution_list = list(zip(validated_df["Institution"].tolist(), validated_df["SubInstitution"].tolist()))
validated_subinstitution_dict = {key: value for key, value in zip(validated_subinstitution_list, validated_df["Category"].tolist())}

#categorize remaining others with the validation list
for index, row in combined_df.iterrows():
    inst_sub = (row["Institution"], row["SubInstitution"])
    if row["Category"] == "Other" and inst_sub in validated_subinstitution_dict.keys():
        print(inst_sub, "    ", validated_subinstitution_dict[inst_sub]) 
        combined_df.at[index, "Category"] = validated_subinstitution_dict[inst_sub]
    elif row["SubInstitution"] == "College":
        combined_df.at[index, "Category"] = "Error"
#number of elements that have yet to be categorized
sorted_combined_df = combined_df.sort_values(by=["Category", "Institution"])
category_counter = Counter(sorted_combined_df['Category'])
num_other = category_counter['Other']
total = len(sorted_combined_df)
print("\n\n Remaining unclassified: ", 100*(num_other/total), "%")

('Agnes Scott College', 'College')      Error
('Allegheny College', 'College')      Error
('Alliant International University', 'Center for Forensic Studies')      Criminal Justice
('Arizona State University', 'College of Public Programs')      Policy And Affairs
('Boston University', 'Professional Schools and Colleges')      Professional Studies
('Boston College', 'Metropolitan College')      Satellite Campus
('Bowling Green State University', 'BGSU Firelands')      Satellite Campus
('Brown University', 'The Undergraduate College')      University College
('Bryn Mawr College', 'Undergraduate College')      University College
('California State University San Bernardino', 'California State University, San Bernardino in Palm Desert')      Satellite Campus
('Capital University', 'Unified College')      Continued Studies
('Carnegie Mellon University', 'Carnegie Mellon West')      Satellite Campus
('Carthage College', 'Carthage College')      Error
('Carthage College', 'Matthews Distributin

In [ ]:
#final df
sorted_combined_df.to_csv(grouped_path, index=False)

category_counter = Counter()
for category_list in combined_df["Category"]:
    categories = [cat.strip() for cat in category_list.split(",") if cat.strip()]
    category_counter.update(categories)

sorted_category = dict(sorted(category_counter.items(), key=lambda item: item[1], reverse=True))
print("\nFinal category counts")
for category, number in sorted_category.items():
    print(f"{category}: {number}")


Final category counts
Business: 483
Education: 399
Graduate: 329
Arts and Sciences: 320
Engineering: 276
Fine Arts: 266
Health Science: 262
Natural Sciences: 226
Nursing: 182
Law: 162
Medical: 130
Social Work: 128
Foundation: 122
Administration: 116
Technology: 104
Humanities: 104
Liberal Arts: 102
Social Sciences: 100
Policy and Affairs: 80
Satellite Campus: 79
Behavioral Science: 78
Pharmacy: 77
Communication: 76
Archictecure: 76
Agriculture: 73
Environmental and Earth Science: 72
Continued Studies: 70
Professional Studies: 58
Information: 56
Error: 55
Dentistry: 52
University College: 45
Computer Science: 39
Religious Studies: 36
Honor College: 31
Veterinary: 27
Journalism: 26
Boards: 15
Physical Therapy and Nutrition: 15
Language and Culture: 13
Hospitality: 11
Hospital: 10
Marine Science: 10
Optometry: 10
Family and Consumer Science: 9
Criminal Justice: 7
Policy And Affairs: 4
Other: 4
Pathology: 2
Culinary: 1
Audiology: 1
Gerontology: 1
Medicine: 1
